In [24]:
import pandas as pd
from haystack.document_stores import OpenSearchDocumentStore
from haystack.schema import Document

In [2]:
ds = OpenSearchDocumentStore()

/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.

In [4]:
with open('./data/processed_reviews.parquet', mode='rb') as file:
    df = pd.read_parquet(file)

In [9]:
df.loc[df['text_s1_len'] > 10].groupby(by='company_name').size().sort_values(ascending=False)

company_name
FERCHAU GmbH                                 499
Netto Marken-Discount AG & Co. KG            439
ALDI SÜD                                     343
Hays                                         287
TEDi GmbH & Co. KG                           285
                                            ... 
Rudnick GmbH                                   1
GEUM.tec GmbH                                  1
Rudolf Dorsch                                  1
GETTO INNENARCHITEKTUR                         1
MWG - Wohnungsgenossenschaft eG Magdeburg      1
Length: 127780, dtype: int64

In [17]:
df.loc[df['company_name'] == 'FERCHAU GmbH', 'profile_uuid', 'job_status'].iloc[0]

'304de031-0ca6-45fe-b57e-99209d358551'

In [18]:
most_texts = '304de031-0ca6-45fe-b57e-99209d358551'

In [20]:
relevant_cols = [
    'text_s1', 
    'text_s1_original', 
    's_1',
    'profile_ebp_status',
    'profile_industry_id',
    'profile_weighted_total']

In [21]:
df.loc[
    (df['profile_uuid'] == most_texts) & 
    (df['text_s1_len'] >= 20) &
    (df['text_s1_language'] == 'de') &
    (df['text_s1_language_confidence'] >= .75), 
    relevant_cols].shape

(288, 6)

In [22]:
df = df.loc[
    (df['profile_uuid'] == most_texts) & 
    (df['text_s1_len'] >= 20) &
    (df['text_s1_language'] == 'de') &
    (df['text_s1_language_confidence'] >= .75), 
    relevant_cols]

In [33]:
# orientstr {‘dict’, ‘list’, ‘series’, ‘split’, ‘tight’, ‘records’, ‘index’}
df.iloc[:2].to_dict(orient='index')

{9632: {'text_s1': 'eingehen zugehen mitarbeiter dankeschön gute laufende projekte bzw anerkennung gehör weiterbildungen dabei heisst slogan bringen',
  'text_s1_original': 'Kein eingehen und zugehen auf die Mitarbeiter. Keine Dankeschön für gute laufende Projekte bzw. keine Anerkennung. Kein Gehör für Weiterbildungen. Dabei heißt es im Slogan "Wir bringen sie weiter". ',
  's_1': 1.0,
  'profile_ebp_status': 'EBP_PRO',
  'profile_industry_id': 38,
  'profile_weighted_total': 4.0774},
 11714: {'text_s1': 'inkompetentes verhalten führungsebene bspw kollegen bevorzugt gegeneinander ausgespielt schlecht gemacht schwächen stark hervorgehoben statt potenzial fördern umdenken nötig',
  'text_s1_original': 'Inkompetentes Verhalten der Führungsebene. Bspw. werden manche Kollegen bevorzugt, gegeneinander ausgespielt oder vor anderen schlecht gemacht. Schwächen werden stark hervorgehoben statt Potenzial zu fördern. Ein Umdenken ist nötig!',
  's_1': 1.0,
  'profile_ebp_status': 'EBP_PRO',
  'pro

In [23]:
df.head()

,text_s1,text_s1_original,s_1,profile_ebp_status,profile_industry_id,profile_weighted_total
9632,eingehen zugehen mitarbeiter dankeschön gute laufende projekte bzw anerkennu...,Kein eingehen und zugehen auf die Mitarbeiter. Keine Dankeschön für gute lau...,1.0,EBP_PRO,38,4.0774
11714,inkompetentes verhalten führungsebene bspw kollegen bevorzugt gegeneinander ...,Inkompetentes Verhalten der Führungsebene. Bspw. werden manche Kollegen bevo...,1.0,EBP_PRO,38,4.0774
20302,gros funktionen vorgesetzter ausübt quasi vorgesetzten beim kunden ausgeführ...,"Das Gros der Funktionen, die ein Vorgesetzter ausübt, wird durch den quasi-V...",5.0,EBP_PRO,38,4.0774
21547,normalerweise gut vorgesetzt zeit mal gewechselt eigentlicher vorgesetzt wur...,Normalerweise gut. Mein Vorgesetzt.er hat sich in der Zeit 3 mal gewechselt....,1.0,EBP_PRO,38,4.0774
24272,merkt vertrieblern tun porsche verkauft bekommt alten twingo ansonsten zulet...,"Man merkt, dass man es mit Vertrieblern zu tun hat. Einem wird ein Porsche v...",1.0,EBP_PRO,38,4.0774


In [77]:
def to_documents(
    data: pd.DataFrame, column_name: str = 's1') -> list[Document]:
    
    result = list()
    
    for index, values in data.to_dict(orient='index').items():
        d = Document(
            id=str(index),
            content=values[f'text_{column_name}_original'],
            meta={
                'stars': values[f's_{column_name[1:]}'], 
                'ebp': values['profile_ebp_status'], 
                'industry': values['profile_industry_id'], 
                'stars_profile': values['profile_weighted_total']})
        
        result.append(d)
        
        # todo: paragraph split
    
    return result

In [78]:
to_documents(df.head())[0]

<Document: {'content': 'Kein eingehen und zugehen auf die Mitarbeiter. Keine Dankeschön für gute laufende Projekte bzw. keine Anerkennung. Kein Gehör für Weiterbildungen. Dabei heißt es im Slogan "Wir bringen sie weiter". ', 'content_type': 'text', 'score': None, 'meta': {'stars': 1.0, 'ebp': 'EBP_PRO', 'industry': 38, 'stars_profile': 4.0774}, 'embedding': None, 'id': '9632'}>

In [40]:
ds.write_documents(to_documents(df))

/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [41]:
from haystack.nodes import BM25Retriever


retriever = BM25Retriever(document_store=ds)

In [42]:
from haystack.nodes import FARMReader

# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


In [43]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [69]:
from pprint import pprint

# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="sind vorgesetzte authentisch?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
pprint(prediction)

/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Inferencing Samples: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.11 Batches/s]

{'answers': [<Answer {'answer': 'Menschen', 'type': 'extractive', 'score': 0.07754438370466232, 'context': 'Fehlentscheidung kommen vor . Vorgesetzte sind eben nur Menschen. Aber in der Regel wird die Strategie der Situation angepasst. Es wird versucht, die ', 'offsets_in_document': [{'start': 56, 'end': 64}], 'offsets_in_context': [{'start': 56, 'end': 64}], 'document_id': '1054356', 'meta': {'stars': 3.0, 'ebp': 'EBP_PRO', 'industry': 38, 'stars_profile': 4.0774}}>,
             <Answer {'answer': 'persönlicher Ansprechpartner eigentlich immer für seinen MA da. Man ist sich perdu', 'type': 'extractive', 'score': 0.03572332486510277, 'context': 'Der Vorgesetzte ist als persönlicher Ansprechpartner eigentlich immer für seinen MA da. Man ist sich perdu. Ich habe allerdings auch schon erlebt, das', 'offsets_in_document': [{'start': 24, 'end': 106}], 'offsets_in_context': [{'start': 24, 'end': 106}], 'document_id': '301451', 'meta': {'stars': 4.0, 'ebp': 'EBP_PRO', 'industry': 38, 'stars_

In [76]:
pprint(prediction['answers'][0].to_dict())

{'answer': 'Menschen',
 'context': 'Fehlentscheidung kommen vor . Vorgesetzte sind eben nur Menschen. '
            'Aber in der Regel wird die Strategie der Situation angepasst. Es '
            'wird versucht, die ',
 'document_id': '1054356',
 'meta': {'ebp': 'EBP_PRO',
          'industry': 38,
          'stars': 3.0,
          'stars_profile': 4.0774},
 'offsets_in_context': [{'end': 64, 'start': 56}],
 'offsets_in_document': [{'end': 64, 'start': 56}],
 'score': 0.07754438370466232,
 'type': 'extractive'}


In [65]:
pprint(df.loc[1054356].to_dict())

{'profile_ebp_status': 'EBP_PRO',
 'profile_industry_id': 38,
 'profile_weighted_total': 4.0774,
 's_1': 3.0,
 'text_s1': 'fehlentscheidung kommen vorgesetzte eben menschen regel strategie '
            'situation angepasst versucht mitarbeiter neue regelung motivieren '
            'änderungen akzeptieren',
 'text_s1_original': 'Fehlentscheidung kommen vor . Vorgesetzte sind eben nur '
                     'Menschen. Aber in der Regel wird die Strategie der '
                     'Situation angepasst. Es wird versucht, die Mitarbeiter '
                     'für die neue Regelung zu motivieren, Änderungen zu '
                     'akzeptieren.'}
